In [15]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torch import nn,optim
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
print(torch.cuda.is_available())  # 应该返回True
print(torch.cuda.device_count())  # 返回可用的GPU数量

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
torch.cuda.amp


True
1
Using device: cuda


<module 'torch.cuda.amp' from 'C:\\Users\\BLKDASH\\.conda\\envs\\py310torch24cuda124\\lib\\site-packages\\torch\\cuda\\amp\\__init__.py'>

In [16]:
# 载入训练集数据，同时把数据转换为tensor，同时下载数据
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 载入测试集数据，同时把数据转换为tensor，同时下载数据
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

# 文件分别为训练集测试集他们分别的数据和标签

In [17]:
# 批次大小
batch_size = 64
# 装载数据集,dataloader为数据的装载器，数据来源于dataset=train_dataset, 大小为batch_size=batch_size，方式为随机打乱
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
# 展示数据
for i,data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [18]:
class Net(nn.Module):
    # 初始化，定义网络结构
    def __init__(self):
        # 初始化nnModule
        super(Net, self).__init__()
        # 定义了一个全连接层，参数为输入值和输出值个数
        # 由于是非线性回归，一层神经网络无法做到，因此需要加入隐藏层
        self.fc1 = nn.Linear(784, 10)
        # 设置激活函数
        self.softmax = nn.Softmax(dim=1) # 对第一个维度计算概率值
        
    # 前向计算，定义网络计算
    def forward(self, x):
        # ([64, 1, 28, 28])->([64,784])
        # 由于第0维度取64，所以-1的位置取28*28
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        outs = self.softmax(x)
        return outs

In [19]:
LR = 0.005
# 定义模型
model = Net()
# model.to(device)
# 定义损失函数,交叉熵损失
cross_entropy = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(model.parameters(), lr=LR)

In [20]:
def train():
    for i,data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        # 交叉熵无需使用独热编码
        loss = cross_entropy(out, labels)
        # 梯度清零，计算梯度，修改权值
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test():
    correct = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device)
        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)
        
        correct += (predicted == labels).sum()
    print(' ',correct.item())
    print("Test acc:{0}".format(correct.item() / len(test_loader.dataset)))
        

In [21]:
for epoch in range(10):
    
    train()
    print('epoch:', epoch,' train over')
    
    test()
    LR = LR * 0.9
    
    
# 一共1w张测试集

epoch: 0  train over
  9210
Test acc:0.921
epoch: 1  train over
  9255
Test acc:0.9255
epoch: 2  train over
  9269
Test acc:0.9269
epoch: 3  train over
  9286
Test acc:0.9286
epoch: 4  train over
  9306
Test acc:0.9306
epoch: 5  train over
  9297
Test acc:0.9297
epoch: 6  train over
  9309
Test acc:0.9309
epoch: 7  train over
  9302
Test acc:0.9302
epoch: 8  train over
  9298
Test acc:0.9298
epoch: 9  train over
  9306
Test acc:0.9306
